# Train Test Split

This notebook creates a train-test split by patient. Once the patients are split, then we randomly sample a subset of each patients lung images to be used for training. 

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
IMAGE_DIR = "/Users/raamis/lung_cancer_detector/data/images/"
LABEL_PATH = "/Users/raamis/lung_cancer_detector/data/labels"

images = os.listdir(IMAGE_DIR)
images = np.core.defchararray.add(IMAGE_DIR,images)

labels = np.core.defchararray.replace(images,".png",".txt")

# Create dataframe of images and labels
patient_df = pd.DataFrame()
patient_df["images"] = images
patient_df["labels"] = labels

patient_names = []
for name in patient_df["images"]:
    id = name.split("/")[-1].split("_")[0]
    patient_names.append(id)

patient_df["patient"] = patient_names
patient_df.head()

,images,labels,patient
0,/Users/raamis/lung_cancer_detector/data/images...,/Users/raamis/lung_cancer_detector/data/images...,G0059
1,/Users/raamis/lung_cancer_detector/data/images...,/Users/raamis/lung_cancer_detector/data/images...,G0044
2,/Users/raamis/lung_cancer_detector/data/images...,/Users/raamis/lung_cancer_detector/data/images...,A0112
3,/Users/raamis/lung_cancer_detector/data/images...,/Users/raamis/lung_cancer_detector/data/images...,A0084
4,/Users/raamis/lung_cancer_detector/data/images...,/Users/raamis/lung_cancer_detector/data/images...,A0249


In [6]:
# Create 80/20 train test split
patients = patient_df["patient"].unique()
train_patients,test_patients = train_test_split(patients,test_size=0.2,train_size=0.8)

In [8]:
print(len(patients))

350


## Sample Train and Test Images

Now that we have split the patients into a train and test set, let's randomly sample images from each patient to create a set of images and labels for training and testing.